In [28]:
import zipfile
import pandas as pd
import numpy as np
import os
import json
from tqdm import tqdm

# Load bankruptcy and non-bankruptcy data 

In [29]:
bank = pd.read_csv("../data/bankrupt.csv", index_col = 0)
bank["bank_status"] = 1
nonbank = pd.read_csv("../data/nonbankrupt.csv", index_col = 0)
nonbank["bank_status"] = 0
all_company = pd.concat([bank, nonbank], ignore_index = True)
all_company["cik"] = all_company["cik"].astype(str).str.zfill(10)
all_company['sic'] = all_company['sic'].astype(str)

In [30]:
print(nonbank.shape)
print(bank.shape)
print(all_company.shape)

(68143, 8)
(320, 8)
(68463, 8)


# Collect financial terms

In [34]:
# find financial number from the companfacts folder 
# Path to your large ZIP file
zip_file_path = '../data/companyfacts.zip'
# Open the ZIP file in read mode
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_contents = set(zip_ref.namelist())
    for value in tqdm(all_company["cik"]):
        # Extract a specific file
        specific_file = 'CIK' + value + '.json'
        if specific_file in zip_contents:
            zip_ref.extract(specific_file, '../data/all_company_data')

def collect(company_json_file, finantial_terms, cik, company_facts):
    tmp = pd.DataFrame(columns = ["filed_date"] + finantial_terms)
    for term in finantial_terms:
        if term in company_json_file['facts']['us-gaap']:
            if term == 'CommonStockSharesIssued':
                units = company_json_file['facts']['us-gaap'][term].get("units", {}).get("shares", [])
            else:
                units = company_json_file['facts']['us-gaap'][term].get("units", {}).get("USD", [])
            for unit in units:
                if unit["form"] == "10-K":
                    filed_date = unit.get("filed")
                    if filed_date in tmp["filed_date"].values:
                        tmp.loc[tmp["filed_date"] == filed_date, term] = unit.get("val")
                    else:
                        tmp = pd.concat([tmp, pd.DataFrame({"filed_date": [filed_date], term: [unit.get("val")]})])
    tmp["cik"] = cik
    company_facts = pd.concat([company_facts, tmp])
    return company_facts

def collect_company_facts(finantial_terms):
    folder_path = '../data/all_company_data'
    company_facts = pd.DataFrame(columns = ["cik", "filed_date"] + finantial_terms)
    
    for file_name in tqdm(os.listdir(folder_path)):
        json_file_path = os.path.join(folder_path, file_name)
        with open(json_file_path, 'r') as json_file:
            company_json_file = json.load(json_file)
            if 'us-gaap' not in company_json_file['facts']:
                continue
            cik = company_json_file.get("cik", "Unknown")
            company_facts = collect(company_json_file, finantial_terms, cik, company_facts)
    
    return company_facts

finantial_terms = ['Assets', 'AssetsCurrent', 'Liabilities', 'LiabilitiesCurrent', \
                   'StockholdersEquity', 'CashAndCashEquivalentsAtCarryingValue', \
                   'NetCashProvidedByUsedInOperatingActivities', \
                   'NetCashProvidedByUsedInFinancingActivities', \
                   'NetCashProvidedByUsedInInvestingActivities', \
                   'NetIncomeLoss', 'OperatingIncomeLoss', \
                   'CommonStockValue', 'CommonStockSharesIssued'
                   ]
company_facts = collect_company_facts(finantial_terms)
company_facts["cik"] = company_facts["cik"].astype(str).str.zfill(10)

100%|███████████████████████████████████████| 6628/6628 [04:10<00:00, 26.50it/s]


# Merge financial terms and companies

In [35]:
merged_data = pd.merge(all_company, company_facts, left_on = ['cik', 'filedate'], right_on = ['cik', 'filed_date'], how = 'inner')
merged_data.drop(columns = ['filed_date'], inplace = True)

In [36]:
np.sum(merged_data.isna())

/Users/haodong/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


cik                                               0
sic                                               0
sic_description                                  92
filedate                                          0
accession_num                                     0
primary_doc                                       0
filelink                                          0
bank_status                                       0
Assets                                          536
AssetsCurrent                                 10611
Liabilities                                    9744
LiabilitiesCurrent                            10705
StockholdersEquity                             3818
CashAndCashEquivalentsAtCarryingValue          5934
NetCashProvidedByUsedInOperatingActivities     7371
NetCashProvidedByUsedInFinancingActivities     7593
NetCashProvidedByUsedInInvestingActivities     9931
NetIncomeLoss                                  3688
OperatingIncomeLoss                           10955
CommonStockV

In [37]:
# Drop lines with NA
merged_data = merged_data.dropna()
print(merged_data.shape)
print("The number of bankruptcy companies:", np.sum(merged_data['bank_status'] == 1))

(13619, 21)
The number of bankruptcy companies: 81


In [38]:
# Drop lines with any finantial term = 0
merged_data = merged_data[(merged_data[finantial_terms] != 0).all(axis = 1)]
print(merged_data.shape)

(12921, 21)


In [39]:
# Calculate finantial ratios
# liquidity ratios
merged_data['current_ratio'] = merged_data['AssetsCurrent'] / merged_data['LiabilitiesCurrent']

# solvency ratios （有看到用debt的，也有看到用liabilities，因为没有debt只能用liabilities）
merged_data['debt_to_equity'] = merged_data['Liabilities'] / merged_data['StockholdersEquity']
merged_data['debt_to_total_assets'] = merged_data['Liabilities'] / merged_data['Assets']

# profitability ratio (分母要用两年的平均值，算的时候只用了当年的)
merged_data['return_on_assets'] = merged_data['NetIncomeLoss'] / merged_data['Assets']
merged_data['return_on_equity'] = merged_data['NetIncomeLoss'] / merged_data['StockholdersEquity']

# other ratios commonly used in previous research （这些似乎不是常见的ratio，pre的时候可能不好讲这些ratios背后的含义，如果跑模型的时候发现这些用处不大可以删掉，如果有用的话pre的时候要么就说是我们查阅文献得到的）
merged_data['ebit_over_assets'] = merged_data['OperatingIncomeLoss'] / merged_data['Assets']
merged_data['current_assets_over_assets'] = merged_data['AssetsCurrent'] / merged_data['Assets']
merged_data['cash_over_assets'] = merged_data['CashAndCashEquivalentsAtCarryingValue'] / merged_data['Assets']
merged_data['cash_operation_over_assets'] = merged_data['NetCashProvidedByUsedInOperatingActivities'] / merged_data['Assets']
merged_data['cash_operation_over_liabilities'] = merged_data['NetCashProvidedByUsedInOperatingActivities'] / merged_data['Liabilities']


In [40]:
print("The final number of bankruptcy companies:", np.sum(merged_data['bank_status'] == 1))

The final number of bankruptcy companies: 72


In [41]:
# Get main industry by grouping the sic code
def map_sic_to_industry(sic_code):
    if 100 <= sic_code <= 999:
        return 'Agriculture, Forestry and Fishing'
    elif 1000 <= sic_code <= 1499:
        return 'Mining'
    elif 1500 <= sic_code <= 1799:
        return 'Construction'
    # Note: 1800-1999 is not used
    elif 2000 <= sic_code <= 3999:
        return 'Manufacturing'
    elif 4000 <= sic_code <= 4999:
        return 'Transportation, Communications, Electric, Gas and Sanitary service'
    elif 5000 <= sic_code <= 5199:
        return 'Wholesale Trade'
    elif 5200 <= sic_code <= 5999:
        return 'Retail Trade'
    elif 6000 <= sic_code <= 6799:
        return 'Finance, Insurance and Real Estate'
    elif 7000 <= sic_code <= 8999:
        return 'Services'
    elif 9100 <= sic_code <= 9729:
        return 'Public Administration'
    elif 9900 <= sic_code <= 9999:
        return 'Nonclassifiable'
    else:
        return 'Unknown'  # For SIC codes that don't fall into any category or missing


merged_data['main_industry'] = merged_data['sic'].astype(float).apply(map_sic_to_industry)

merged_data['filedate'] = pd.to_datetime(merged_data['filedate']) 
merged_data['year'] = merged_data['filedate'].dt.year
merged_data

,cik,sic,sic_description,filedate,accession_num,primary_doc,filelink,bank_status,Assets,AssetsCurrent,Liabilities,LiabilitiesCurrent,StockholdersEquity,CashAndCashEquivalentsAtCarryingValue,NetCashProvidedByUsedInOperatingActivities,NetCashProvidedByUsedInFinancingActivities,NetCashProvidedByUsedInInvestingActivities,NetIncomeLoss,OperatingIncomeLoss,CommonStockValue,CommonStockSharesIssued,current_ratio,debt_to_equity,debt_to_total_assets,return_on_assets,return_on_equity,ebit_over_assets,current_assets_over_assets,cash_over_assets,cash_operation_over_assets,cash_operation_over_liabilities,main_industry,year
3,0001606180,8093.0,"Services-Specialty Outpatient Facilities, NEC",2019-04-15,0001564590-19-011552,aac-10k_20181231.htm,https://www.sec.gov/Archives/edgar/data/160618...,1,452277000,63964000,410004000,361445000,64413000,5409000,-28854000,105221000,-84776000,-59404000,-37538000,25000,24573679,0.176967,6.365237,0.906533,-0.131344,-0.922236,-0.082998,0.141427,0.011959,-0.063797,-0.070375,Services,2019
4,0000002034,5122.0,"Wholesale-Drugs, Proprietaries & Druggists' Su...",2018-09-28,0001144204-18-051414,tv501271_10k.htm,https://www.sec.gov/Archives/edgar/data/2034/0...,1,767024000,502627000,671739000,302518000,95285000,100874000,101806000,-48863000,-8281000,-34726000,-275012000,308000,30787000,1.661478,7.049787,0.875773,-0.045274,-0.364444,-0.358544,0.655295,0.131513,0.132729,0.151556,Wholesale Trade,2018
7,0000003116,2834.0,Pharmaceutical Preparations,2020-02-26,0001628280-20-002314,akorn10k12312019.htm,https://www.sec.gov/Archives/edgar/data/3116/0...,1,1288639000,480047000,1054347000,985587000,234292000,144804000,-36919000,-12928000,-30402000,-80660000,-43980000,595521000,126145832,0.487067,4.500141,0.818186,-0.062593,-0.344271,-0.034129,0.372522,0.11237,-0.02865,-0.035016,Manufacturing,2020
8,0001376610,1040.0,Gold and Silver Ores,2015-03-27,0001376610-15-000004,anv-2014123110xk.htm,https://www.sec.gov/Archives/edgar/data/137661...,1,941238000,298847000,663680000,592376000,277558000,7575000,832000,-8853000,-65874000,-461219000,-480122000,126000,126193336,0.504489,2.39114,0.705114,-0.490013,-1.661703,-0.510096,0.317504,0.008048,0.000884,0.001254,Mining,2015
20,0001405073,1311.0,Crude Petroleum & Natural Gas,2019-03-18,0001564590-19-008275,arex-10k_20181231.htm,https://www.sec.gov/Archives/edgar/data/140507...,1,1084647000,16225000,495402000,21077000,589245000,22000,34744000,8021000,-42764000,868000,1209000,950000,95030569,0.769796,0.84074,0.45674,0.0008,0.001473,0.001115,0.014959,0.00002,0.032033,0.070133,Mining,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48415,0001403568,5990.0,"Retail-Retail Stores, NEC",2020-03-27,0001558370-20-003272,ulta-20200201x10k11b4b7.htm,https://www.sec.gov/Archives/edgar/data/140356...,0,4863872000,2055317000,2961778000,1137261000,1902094000,392325000,1101293000,-646739000,-471480000,222719000,287756000,573000,57285000,1.807252,1.557114,0.608934,0.04579,0.117091,0.059162,0.422568,0.080661,0.226423,0.371835,Retail Trade,2020
48416,0001403568,5990.0,"Retail-Retail Stores, NEC",2019-04-02,0001558370-19-002739,ulta-20190202x10k.htm,https://www.sec.gov/Archives/edgar/data/140356...,0,3191172000,1914861000,1370954000,823736000,1820218000,409251000,956127000,-609214000,-215107000,214674000,281210000,592000,59232000,2.324605,0.753181,0.429608,0.067271,0.117939,0.088121,0.600049,0.128245,0.299616,0.697417,Retail Trade,2019
48417,0001403568,5990.0,"Retail-Retail Stores, NEC",2018-04-03,0001558370-18-002733,ulta-20180203x10k.htm,https://www.sec.gov/Archives/edgar/data/140356...,0,2908687000,1693743000,1134470000,642166000,1774217000,277445000,779366000,-356217000,-530714000,208173000,254419000,614000,61441000,2.637547,0.63942,0.390028,0.071569,0.117332,0.087469,0.582305,0.095385,0.267944,0.686987,Retail Trade,2018
48422,0001403568,5990.0,"Retail-Retail Stores, NEC",2013-04-03,0001193125-13-140116,d466694d10k

In [43]:
merged_data.to_csv("../data/data.csv", index = False)